In [26]:
import numpy as np
import pandas as pd

In [142]:
consumer_complaints = pd.read_csv("consumer_complaints.csv")


/var/folders/h6/9qtkdjdj5nj72hzhyqcrqhcw0000gn/T/ipykernel_37283/597542118.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  consumer_complaints = pd.read_csv("consumer_complaints.csv")


In [143]:
state_populations = pd.read_csv("PEPPOP2021.NST_EST2021_POP-2022-09-29T183532.csv")

In [146]:
#Data ends on this date
print(max(consumer_complaints["date_sent_to_company"]))

12/31/2015


In [147]:
#Data starts on this date
print(min(consumer_complaints["date_sent_to_company"]))

01/01/2013


In [148]:
#I took this list and map from JeffPaine/us_state_abbreviations.py 
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [149]:
count_per_state = []

for state in states:
    count_per_state.append(sum(consumer_complaints.state == state))
total = np.array(count_per_state).sum()
print("Count per state, based on states list above")
print(count_per_state)
print("Percentage per state, based on states list above")
print(count_per_state/total)

Count per state, based on states list above
[638, 5635, 2385, 12348, 81700, 9495, 6445, 3224, 2917, 53673, 24548, 1937, 2290, 1935, 19624, 6139, 2694, 3992, 5403, 11105, 17703, 1942, 14486, 6424, 7233, 2570, 893, 15501, 476, 1742, 2826, 22408, 2776, 6779, 38266, 17380, 3630, 6604, 20015, 1899, 7013, 757, 8523, 41352, 2951, 18150, 943, 11554, 6125, 1457, 551]
Percentage per state, based on states list above
[0.00116199 0.01026307 0.00434382 0.02248951 0.14880085 0.01729332
 0.01173833 0.0058719  0.00531275 0.09775506 0.04470946 0.00352787
 0.00417079 0.00352423 0.03574135 0.01118101 0.0049066  0.00727066
 0.00984053 0.02022562 0.03224261 0.00353698 0.02638347 0.01170008
 0.01317352 0.00468076 0.00162643 0.02823209 0.00086694 0.00317272
 0.00514702 0.04081187 0.00505595 0.01234665 0.06969417 0.03165433
 0.00661135 0.01202792 0.03645348 0.00345866 0.01277283 0.00137873
 0.01552301 0.07531472 0.00537468 0.03305674 0.00171749 0.02104339
 0.01115551 0.00265365 0.00100354]


In [150]:
#Most common product and subproduct complaint
print("Most common product complaint: ")
print(consumer_complaints["product"].mode())

print("Most common sub-product complaint: ")
print(consumer_complaints["sub_product"].mode())



Most common product complaint: 
0    Mortgage
Name: product, dtype: object
Most common sub-product complaint: 
0    Other mortgage
Name: sub_product, dtype: object


In [184]:
#create testing and training sets
rand_int = np.random.rand(len(consumer_complaints)) < 0.8
train = consumer_complaints[rand_int]
test = consumer_complaints[~rand_int]
print(test.head())

   date_received          product                             sub_product  \
6     08/30/2013      Credit card                                     NaN   
9     09/17/2013         Mortgage  Conventional adjustable mortgage (ARM)   
10    08/30/2013  Debt collection   Other (i.e. phone, health club, etc.)   
13    08/30/2013         Mortgage                          Other mortgage   
14    09/10/2013  Debt collection                             Credit card   

                                       issue                  sub_issue  \
6              Credit line increase/decrease                        NaN   
9   Loan modification,collection,foreclosure                        NaN   
10     Cont'd attempts collect debt not owed              Debt was paid   
13  Application, originator, mortgage broker                        NaN   
14                     Communication tactics  Called outside of 8am-9pm   

   consumer_complaint_narrative company_public_response  \
6                          

In [200]:
#Arrays of unique products, companies, states, and company responses
unique_product = consumer_complaints["product"].unique()
unique_company = consumer_complaints["company"].unique()
unique_state = consumer_complaints["state"].unique()
unique_company_response = consumer_complaints["company_response_to_consumer"].unique()


In [201]:
#probabilities of each unique response 
prob_responses = []
for response in unique_company_response:
    prob_responses.append(sum(train["company_response_to_consumer"] == response))
prob_responses = np.array(prob_responses)
prob_responses = prob_responses/sum(prob_responses)


In [202]:
#probabilities of each unique state
prob_states = []
for state in unique_state:
    prob_states.append(sum(train["state"] == state))
prob_states = np.array(prob_states)
prob_states = prob_states/sum(prob_states)

In [203]:
#probabilities of each unique product
prob_product = []
for product in unique_product:
    prob_product.append(sum(train["product"] == product))
prob_product = np.array(prob_product)
prob_product = prob_product/sum(prob_product)

In [204]:
#probabilities of each unique company
#This has a long runtime unfortunately, due to the sheer size of the number of unique companies
prob_company = []
for company in unique_company:
    prob_company.append(sum(train["company"] == company))
prob_company = np.array(prob_company)
prob_company = prob_company/sum(prob_company)


In [205]:
#probability of state given company response
#Here, we iterate through every response and and seperate into new dataframes
#Then, we iterate through each state and get the probability of seeing it with the current response
prob_state_given_response = []

for response in unique_company_response:
    curr_df = train.loc[train["company_response_to_consumer"] == response]
    curr_total = sum(train["company_response_to_consumer"] == response)
    curr_probs = []
    for state in unique_state:
        curr_probs.append((sum(curr_df["state"] == state))/curr_total)
    prob_state_given_response.append(curr_probs)

In [206]:
#probability of product given company response
#We do the same thing as above 
prob_product_given_response = []

for response in unique_company_response:
    curr_df = train.loc[train["company_response_to_consumer"] == response]
    curr_total = sum(train["company_response_to_consumer"] == response)
    curr_probs = []
    for product in unique_product:
        curr_probs.append((sum(curr_df["product"] == product))/curr_total)
    prob_product_given_response.append(curr_probs)


In [207]:
#probability of company given company response
#This also has a long run time because of the number of unique companies
#We do the same thing as above
prob_company_given_response = []

for response in unique_company_response:
    curr_df = train.loc[train["company_response_to_consumer"] == response]
    curr_total = sum(train["company_response_to_consumer"] == response)
    curr_probs = []
    for company in unique_company:
        curr_probs.append((sum(curr_df["company"] == company))/curr_total)
    prob_company_given_response.append(curr_probs)

In [208]:
#Convert to a list 
unique_product = unique_product.tolist()
unique_company = unique_company.tolist()
unique_state = unique_state.tolist()

In [209]:
#Naive Bayes Classifier 
def naiveBayesClassifier(product, company, state):
    prob_of_response = []
    i = 0
    #Index of product, company and state
    p_index = unique_product.index(product)
    c_index = unique_company.index(company)
    s_index = unique_state.index(state)
    #Iterate through responses 
    for response in unique_company_response:
        #Probability
        curr_result = prob_responses[i] * prob_product_given_response[i][p_index] * prob_state_given_response[i][s_index] * prob_company_given_response[i][c_index]
        prob_of_response.append(curr_result)
        i += 1
    #Find max index
    max_index = prob_of_response.index(max(prob_of_response))
    #return value 
#     print(unique_company_response[max_index])
    return unique_company_response[max_index]
        


In [211]:
#Testing naive bayes classifier 
count_correct = 0
count_total = 0
#print(test.iloc[[3]])
for i in range(len(test)):
    count_total += 1
    curr_row = test.iloc[i]
    count_correct += (naiveBayesClassifier(curr_row.iat[1], curr_row.iat[7], curr_row.iat[8]) == curr_row.iat[14])
print("percentage of correctness: ")
print(count_correct/count_total)

percentage of correctness: 
0.7121438336451288
